In [17]:
import pandas as pd

# Replace 'your_file.csv' with the path to your file
file_path = './Synthetic-Persona-Chat/data/Synthetic-Persona-Chat_test.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dfframe
print(df.head())

                                     user 1 personas  \
0  I just bought a brand new house.\nI like to da...   
1  I am an old man.\nI have a wheelchair that is ...   
2  I met my best friend in kindergarten.\nI am of...   
3  I love having facial hair.\nI hope to retire s...   
4  I love watching movies and tv.\nI have a husba...   

                                     user 2 personas  \
0  I love to meet new people.\nI have a turtle na...   
1  I drive a ford pickup truck.\nI am very conser...   
2  I have two dogs and one cat.\nI work as a nurs...   
3  I am happy being single and alone.\nI only dri...   
4  I am afraid of heights.\nI love animals and ha...   

                         Best Generated Conversation  
0  User 1: Hi, I'm [User 1's name]. What's your n...  
1  User 1: Hi there!\nUser 2: Hello.\nUser 1: I'm...  
2  User 1: Hi! What do you do for work?\nUser 2: ...  
3  User 1: How are you doing today?\nUser 2: I'm ...  
4  User 1: Hi there!\nUser 2: Hi there!\nUser 1: ..

In [103]:
data_list = df.to_dict(orient='records')
data_list[0:5]

[{'user 1 personas': 'I just bought a brand new house.\nI like to dance at the club.\nI run a dog obedience school.\nI have a big sweet tooth.\nI like taking and posting selkies.',
  'user 2 personas': 'I love to meet new people.\nI have a turtle named timothy.\nMy favorite sport is ultimate frisbee.\nMy parents are living in bora bora.\nAutumn is my favorite season.',
  'Best Generated Conversation': "User 1: Hi, I'm [User 1's name]. What's your name?\nUser 2: Hi, I'm [User 2's name]. It's nice to meet you.\nUser 1: Nice to meet you too. What are you interested in?\nUser 2: I like to meet new people, play ultimate frisbee, and spend time with my family.\nUser 1: Those are all great things to be interested in. I like to dance at the club, run a dog obedience school, and eat a lot of sweets.\nUser 2: That's interesting. I've never met anyone who runs a dog obedience school before.\nUser 1: It's a lot of fun. I get to work with dogs every day and help them learn how to behave.\nUser 2: T

In [104]:
data_list = [{"persona":instance["user 1 personas"],"conversation":instance["Best Generated Conversation"]} for instance in data_list]

In [55]:
from transformers import BartTokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
add_special_tokens = {'additional_special_tokens': ['<query>', '<response>', '<latent>', '<persona>']}
tokenizer.add_special_tokens(add_special_tokens)

4

In [56]:
args = {
   "num_latent": 10,
   "num_latent2": 10,
}

In [57]:
encoded_tokens = tokenizer.encode(add_special_tokens['additional_special_tokens'])
args['bos'], args["query"], args["response"], args["latent"], args["persona"], args["eos"] = encoded_tokens
args["pad"] = 1

In [58]:
args

{'num_latent': 10,
 'num_latent2': 10,
 'bos': 0,
 'query': 50265,
 'response': 50266,
 'latent': 50267,
 'persona': 50268,
 'eos': 2,
 'pad': 1}

In [105]:
for i, instance in enumerate(data_list):
    try:
        instance["persona"] = instance["persona"].split("\n")
        instance["conversation"] = instance["conversation"].split("\n")
        instance["persona"] = [p.strip() for p in instance["persona"] if p.strip()]
        instance["conversation"] = [c.strip() for c in instance["conversation"] if c.strip()]

        instance["conversation"] = [c.replace("User 1:", "") for c in instance["conversation"]]
        instance["conversation"] = [c.replace("User 2:", "").strip() for c in instance["conversation"]]
    except Exception as e:
        print("Error in instance:", instance)
        print("Error in instance index:", i)
        print("Exception:", e)
        continue

Error in instance: {'persona': ['I love playing video games.', 'Hey there my name is jordan and i am a veterinarian.', 'I am also a musician on the weekends.', 'Love to read drama books.'], 'conversation': nan}
Error in instance index: 510
Exception: 'float' object has no attribute 'split'


In [106]:
data_list[510]

{'persona': ['I love playing video games.',
  'Hey there my name is jordan and i am a veterinarian.',
  'I am also a musician on the weekends.',
  'Love to read drama books.'],
 'conversation': nan}

In [108]:
data_list.pop(510)

{'persona': ['I run to relieve stress.',
  'I am learning jujitsu but am still new to it.',
  'I am taking college and hoping to be a teacher.',
  'Poker is my favorite card game.',
  'I am a huge ed sheeran fan.'],
 'conversation': ['What do you do for fun?',
  "I love to read fantasy books. I've been reading the Lord of the Rings series recently.",
  "Oh, I love the Lord of the Rings! I've read the books and seen the movies.",
  'Me too! The books are so much better than the movies, though.',
  'I agree. The books have so much more detail and world-building.',
  "What's your favorite part of the books?",
  "I love the characters. They're so well-developed and relatable.",
  "Me too! I also love the story. It's so epic and inspiring.",
  "I agree. It's one of my favorite books of all time.",
  "Me too! I'm so glad I met someone who loves the Lord of the Rings as much as I do.",
  "Me too! It's always nice to meet a fellow fan.",
  'So, what other hobbies do you have?',
  'I also like 

In [110]:
data_list[0]

{'persona': ['I just bought a brand new house.',
  'I like to dance at the club.',
  'I run a dog obedience school.',
  'I have a big sweet tooth.',
  'I like taking and posting selkies.'],
 'conversation': ["Hi, I'm [User 1's name]. What's your name?",
  "Hi, I'm [User 2's name]. It's nice to meet you.",
  'Nice to meet you too. What are you interested in?',
  'I like to meet new people, play ultimate frisbee, and spend time with my family.',
  'Those are all great things to be interested in. I like to dance at the club, run a dog obedience school, and eat a lot of sweets.',
  "That's interesting. I've never met anyone who runs a dog obedience school before.",
  "It's a lot of fun. I get to work with dogs every day and help them learn how to behave.",
  'That sounds like a great job. I love dogs.',
  "Me too! They're the best.",
  "I have a turtle named Timothy. He's my best friend.",
  'I love turtles! I used to have a turtle when I was a kid.',
  "That's awesome. What was its name?"

In [ ]:
data_ids = [
    {
        "persona":[tokenizer.encode(persona) for persona in instance['persona']],
        "conversation":[tokenizer.encode(utterance) for utterance in instance['conversation']]
    }
    
    for instance in data_list
]

In [113]:
for dat in data_ids:
    for persona in dat['persona']:
       persona.pop(0)

In [118]:
for dat in data_ids:
    for i,utter in enumerate(dat['conversation']):
        if i % 2 == 0:
            utter[0] = args["query"]
        else:
            utter[0] = args["response"]

In [119]:
data_ids[0]

{'persona': [[100, 95, 2162, 10, 1518, 92, 790, 4, 2],
  [100, 101, 7, 3836, 23, 5, 950, 4, 2],
  [100, 422, 10, 2335, 41227, 334, 4, 2],
  [100, 33, 10, 380, 4045, 13495, 4, 2],
  [100, 101, 602, 8, 6016, 842, 462, 16731, 4, 2]],
 'conversation': [[50265,
   30086,
   6,
   38,
   437,
   646,
   44518,
   112,
   18,
   766,
   8174,
   653,
   18,
   110,
   766,
   116,
   2],
  [50266,
   30086,
   6,
   38,
   437,
   646,
   44518,
   132,
   18,
   766,
   8174,
   85,
   18,
   2579,
   7,
   972,
   47,
   4,
   2],
  [50265, 41541, 7, 972, 47, 350, 4, 653, 32, 47, 2509, 11, 116, 2],
  [50266,
   100,
   101,
   7,
   972,
   92,
   82,
   6,
   310,
   7017,
   6664,
   354,
   14650,
   6,
   8,
   1930,
   86,
   19,
   127,
   284,
   4,
   2],
  [50265,
   11195,
   32,
   70,
   372,
   383,
   7,
   28,
   2509,
   11,
   4,
   38,
   101,
   7,
   3836,
   23,
   5,
   950,
   6,
   422,
   10,
   2335,
   41227,
   334,
   6,
   8,
   3529,
   10,
   319,
   9,
   29

In [128]:
def get_history_windows(data, window_size=5):
    history_windows = []

    for i in range(len(data)):
        if data[i][0] == args["query"]:
            window = []
            # add the query itself first
            window.append(data[i])

            # move backwards and collect previous turns (up to window_size)
            j = i - 1
            while j >= 0 and len(window) < window_size:
                window.insert(0, data[j])
                j -= 1
            
            history_windows.append(window)

    return history_windows

In [129]:
from itertools import chain

for l in get_history_windows(data_ids[0]['conversation']):
    # print(l)
    ids = list(chain.from_iterable(l))
    print(ids)
    print(tokenizer.decode(ids))
    # break

[50265, 30086, 6, 38, 437, 646, 44518, 112, 18, 766, 8174, 653, 18, 110, 766, 116, 2]
<query> Hi, I'm [User 1's name]. What's your name?</s>
[50265, 30086, 6, 38, 437, 646, 44518, 112, 18, 766, 8174, 653, 18, 110, 766, 116, 2, 50266, 30086, 6, 38, 437, 646, 44518, 132, 18, 766, 8174, 85, 18, 2579, 7, 972, 47, 4, 2, 50265, 41541, 7, 972, 47, 350, 4, 653, 32, 47, 2509, 11, 116, 2]
<query> Hi, I'm [User 1's name]. What's your name?</s> <response> Hi, I'm [User 2's name]. It's nice to meet you.</s> <query> Nice to meet you too. What are you interested in?</s>
[50265, 30086, 6, 38, 437, 646, 44518, 112, 18, 766, 8174, 653, 18, 110, 766, 116, 2, 50266, 30086, 6, 38, 437, 646, 44518, 132, 18, 766, 8174, 85, 18, 2579, 7, 972, 47, 4, 2, 50265, 41541, 7, 972, 47, 350, 4, 653, 32, 47, 2509, 11, 116, 2, 50266, 100, 101, 7, 972, 92, 82, 6, 310, 7017, 6664, 354, 14650, 6, 8, 1930, 86, 19, 127, 284, 4, 2, 50265, 11195, 32, 70, 372, 383, 7, 28, 2509, 11, 4, 38, 101, 7, 3836, 23, 5, 950, 6, 422, 10, 23

In [132]:
tokenizer.decode(list(chain.from_iterable(data_ids[0]['persona'])))


'I just bought a brand new house.</s>I like to dance at the club.</s>I run a dog obedience school.</s>I have a big sweet tooth.</s>I like taking and posting selkies.</s>'

In [ ]:
# args['bos'], args["query"], args["response"], args["latent"], args["persona"], args["eos"] 

In [138]:
train_data = [
    [args["latent"],args["persona"]] + list(chain.from_iterable(instance['persona'])) + list(chain.from_iterable(instance['conversation']))
    for instance in data_ids
]